In [ ]:
!pip install langchain
!pip install PyPDF2
!pip install qdrant_client
!pip install langchain_openai
!pip install pypdf
!pip install openai

In [ ]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import ConversationalRetrievalChain
from qdrant_client import QdrantClient,models
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"]=""
os.environ["QDRANT_URL"]=""
os.environ["QDRANT_COLLECTION_NAME"]=""
os.environ["QDRANT_API_KEY"]=""

In [ ]:
#create new cluseter in qdrant
record=0

connection = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key=os.environ.get("QDRANT_API_KEY"),
)



info = connection.get_collection(collection_name=os.environ.get("QDRANT_COLLECTION_NAME"))

print("Collection info:", info)
for get_info in info:
  print(get_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=91 indexed_vectors_count=0 points_count=91 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}
('sta

In [ ]:
def get_embeddings():
  return OpenAIEmbeddings(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    chunk_size=1
  )

In [ ]:
from langchain_openai import ChatOpenAI
def get_chat_model():
  return ChatOpenAI(temperature=0)

In [ ]:
def ask_question_with_context(qa, question, chat_history):

    query = ""
    result = qa({"question": question, "chat_history": chat_history})
    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

In [ ]:
from langchain.prompts import PromptTemplate
def main():

    embeddings = get_embeddings()

    llm = get_chat_model()

    vector_store = Qdrant(
        client=connection,
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        embeddings=embeddings,
    )
    QUESTION_PROMPT = PromptTemplate.from_template("""你是護士，只能回答健康檢查的問題，其他問題一律不回答。並以繁體中文回答問題。""")

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k": 3}),
        return_source_documents=True,
        condense_question_llm = llm,
        condense_question_prompt=QUESTION_PROMPT,
        verbose=False
    )


    chat_history = []
    while True:
        query = input('you: ')
        if query == 'q':
            break
        chat_history = ask_question_with_context(qa, query, chat_history)


if __name__ == "__main__":
    main()


you: 什麼是低血壓
answer: 低血壓是指血液在血管中流動時对血管壁施加的压力过低。通常，成年人的理想血压应该控制在舒张压低于80mmHg和舒张压低于120mmHg。低血压可能会导致头晕、乏力、虚弱等症状。
you: q
